In [2]:
pip install geopandas

     |████████████████████████████████| 1.0MB 7.6MB/s 
     |████████████████████████████████| 15.3MB 283kB/s 
     |████████████████████████████████| 6.5MB 15.0MB/s 


In [3]:
pip install geopy

In [6]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [7]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [8]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Champ de Mars, Rue Saint-Dominique, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude = 48.85614465, Longitude = 2.297820393322227


In [9]:
df = pd.read_csv("/content/illinoisCrime.csv")
df.head()

,Incident,Date reported,Time reported,Date occurred,Time occurred,General Location,Crime Description,Disposition,Latitude,Longitude
0,UU2105449,4/7/2021,8:41,4/6/2021,17:33,"805 W GREEN ST, URBANA",BURGLARY FROM MOTOR VEHICLE,REPORTED TO OTHER AGENCY,NaN,NaN
1,IF2100208,4/7/2021,1:02,4/7/2021,1:02,SECOND ST/GREEN ST,LIQUOR‐UNDERAGE,ADMIN CLOSED,NaN,NaN
2,UU2105371,4/6/2021,8:06,4/5/2021,15:19,"814 STOUGHTON ST, URBANA",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN
3,CC2109632,4/6/2021,8:06,4/5/2021,18:51,"55 E HEALEY ST, CHAMPAIGN",BURGLARY,REPORTED TO OTHER AGENCY,NaN,NaN
4,CC2109610,4/6/2021,8:06,4/5/2021,15:05,"1007 LOCUST ST, CHAMPAIGN",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN


In [10]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['General Location'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
df.head()

,Incident,Date reported,Time reported,Date occurred,Time occurred,General Location,Crime Description,Disposition,Latitude,Longitude,location,point
0,UU2105449,4/7/2021,8:41,4/6/2021,17:33,"805 W GREEN ST, URBANA",BURGLARY FROM MOTOR VEHICLE,REPORTED TO OTHER AGENCY,NaN,NaN,"(805, West Green Street, West Urbana, Urbana, ...","(40.11052589795918, -88.21793553061224, 0.0)"
1,IF2100208,4/7/2021,1:02,4/7/2021,1:02,SECOND ST/GREEN ST,LIQUOR‐UNDERAGE,ADMIN CLOSED,NaN,NaN,"(Green Street, Foxes, Kingsley, East Worldham,...","(51.1356073, -0.9040264, 0.0)"
2,UU2105371,4/6/2021,8:06,4/5/2021,15:19,"814 STOUGHTON ST, URBANA",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN,None,None
3,CC2109632,4/6/2021,8:06,4/5/2021,18:51,"55 E HEALEY ST, CHAMPAIGN",BURGLARY,REPORTED TO OTHER AGENCY,NaN,NaN,"(55, East Healey Street, Champaign, Champaign ...","(40.111342708184466, -88.23914910185225, 0.0)"
4,CC2109610,4/6/2021,8:06,4/5/2021,15:05,"1007 LOCUST ST, CHAMPAIGN",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN,"(Locust Street, Pesotum, Champaign County, Ill...","(39.9115168, -88.2716449, 0.0)"


In [12]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

,Incident,Date reported,Time reported,Date occurred,Time occurred,General Location,Crime Description,Disposition,Latitude,Longitude,location,point,latitude,longitude,altitude
0,UU2105449,4/7/2021,8:41,4/6/2021,17:33,"805 W GREEN ST, URBANA",BURGLARY FROM MOTOR VEHICLE,REPORTED TO OTHER AGENCY,NaN,NaN,"(805, West Green Street, West Urbana, Urbana, ...","(40.11052589795918, -88.21793553061224, 0.0)",40.110526,-88.217936,0.0
1,IF2100208,4/7/2021,1:02,4/7/2021,1:02,SECOND ST/GREEN ST,LIQUOR‐UNDERAGE,ADMIN CLOSED,NaN,NaN,"(Green Street, Foxes, Kingsley, East Worldham,...","(51.1356073, -0.9040264, 0.0)",51.135607,-0.904026,0.0
2,UU2105371,4/6/2021,8:06,4/5/2021,15:19,"814 STOUGHTON ST, URBANA",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN,None,None,NaN,NaN,NaN
3,CC2109632,4/6/2021,8:06,4/5/2021,18:51,"55 E HEALEY ST, CHAMPAIGN",BURGLARY,REPORTED TO OTHER AGENCY,NaN,NaN,"(55, East Healey Street, Champaign, Champaign ...","(40.111342708184466, -88.23914910185225, 0.0)",40.111343,-88.239149,0.0
4,CC2109610,4/6/2021,8:06,4/5/2021,15:05,"1007 LOCUST ST, CHAMPAIGN",THEFT,REPORTED TO OTHER AGENCY,NaN,NaN,"(Locust Street, Pesotum, Champaign County, Ill...","(39.9115168, -88.2716449, 0.0)",39.911517,-88.271645,0.0


In [13]:
df.columns

Index(['Incident', 'Date reported', 'Time reported', 'Date occurred',
       'Time occurred', 'General Location', 'Crime Description', 'Disposition',
       'Latitude', 'Longitude', 'location', 'point', 'latitude', 'longitude',
       'altitude'],
      dtype='object')

In [14]:
df = df.drop(['General Location', 'Latitude', 'Longitude'], axis=1)

In [15]:
df.head()

,Incident,Date reported,Time reported,Date occurred,Time occurred,Crime Description,Disposition,location,point,latitude,longitude,altitude
0,UU2105449,4/7/2021,8:41,4/6/2021,17:33,BURGLARY FROM MOTOR VEHICLE,REPORTED TO OTHER AGENCY,"(805, West Green Street, West Urbana, Urbana, ...","(40.11052589795918, -88.21793553061224, 0.0)",40.110526,-88.217936,0.0
1,IF2100208,4/7/2021,1:02,4/7/2021,1:02,LIQUOR‐UNDERAGE,ADMIN CLOSED,"(Green Street, Foxes, Kingsley, East Worldham,...","(51.1356073, -0.9040264, 0.0)",51.135607,-0.904026,0.0
2,UU2105371,4/6/2021,8:06,4/5/2021,15:19,THEFT,REPORTED TO OTHER AGENCY,None,None,NaN,NaN,NaN
3,CC2109632,4/6/2021,8:06,4/5/2021,18:51,BURGLARY,REPORTED TO OTHER AGENCY,"(55, East Healey Street, Champaign, Champaign ...","(40.111342708184466, -88.23914910185225, 0.0)",40.111343,-88.239149,0.0
4,CC2109610,4/6/2021,8:06,4/5/2021,15:05,THEFT,REPORTED TO OTHER AGENCY,"(Locust Street, Pesotum, Champaign County, Ill...","(39.9115168, -88.2716449, 0.0)",39.911517,-88.271645,0.0


In [16]:
df = df[pd.notnull(df["latitude"])]

In [17]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)

In [18]:
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [19]:
map1.save("map1.html")

In [20]:
folium_map = folium.Map(location=[59.338315,18.089960],
                        zoom_start=12,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [21]:
folium_map.save("folium_map.html")

In [22]:
df.to_csv ('geolocation_illinoisCrime.csv', index = False, header=True)